In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [3]:
qp = QuadraticProgram()
qp.binary_var('e01')
qp.binary_var('e02')
qp.binary_var('e10')
qp.binary_var('e12')
qp.binary_var('e20')
qp.binary_var('e21')
qp.maximize(linear=[4, 8, 0, 8, 0, 4])
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1

Binaries
 x01 x02 x10 x12 x20 x21
End



In [4]:
qp.linear_constraint(linear={'e01':1, 'e02':1}, sense='==', rhs=1, name='start')
qp.linear_constraint(linear={'e01':1, 'e21':1}, sense='<=', rhs=1, name='one1')
qp.linear_constraint(linear={'e02':1, 'e12':1}, sense='<=', rhs=1, name='one2')
qp.linear_constraint(linear={'e01':4, 'e02':6,
                             'e10':2, 'e12':5,
                             'e20':4, 'e21':5}, sense='<=', rhs=9, name='deadline')
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To
 start: x01 + x02 = 1
 one1: x01 + x21 <= 1
 one2: x02 + x12 <= 1
 deadline: 4 x01 + 6 x02 + 2 x10 + 5 x12 + 4 x20 + 5 x21 <= 9

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1

Binaries
 x01 x02 x10 x12 x20 x21
End



In [5]:
aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

In [6]:
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)

In [7]:
exact_result = exact.solve(qp)
print(exact_result)

optimal function value: 12.0
optimal value: [1. 0. 0. 1. 0. 0.]
status: SUCCESS


In [8]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result)

optimal function value: 12.0
optimal value: [1. 0. 0. 1. 0. 0.]
status: SUCCESS
